<h1>Exercise 3: The Resume Agent</h1>

### The Challenge
Build an agent that constructs a profile description step-by-step.

**Requirements:**
1. State includes `name`, `age`, `skills` (list), and `result`.
2. **Node 1**: Initialize result with "[Name], welcome to the system! "
3. **Node 2**: Append "You are [Age] years old. "
4. **Node 3**: Append "You have skills in: [Skill1], [Skill2]..."
5. Connect them in sequence: 1 -> 2 -> 3.

### The Solution
Let's build this pipeline.

In [ ]:
from typing import TypedDict, List
from langgraph.graph import StateGraph

### Step 1: State
Defining the data structure. Note `skills` is a `List[str]`.

In [ ]:
class AgentState(TypedDict):
    name: str
    age: int
    skills: List[str]
    result: str

### Step 2: The Nodes
We break the job into three small, focused functions.

In [ ]:
def firstNode(state: AgentState) -> AgentState:
    """ This node adds the name to the result"""
    state["result"] = f"{state["name"]}, welcome to the system! "
    return state

In [ ]:
def secondNode(state: AgentState) -> AgentState:
    """This node adds the age to the result""" 
    state["result"] = state["result"] + f"You are {state["age"]} years old ! "
    return state

In [ ]:
def thirdNode(state: AgentState) -> AgentState:
    """This node adds the skills to the result""" 
    state["result"] = state["result"] + "You have skills in: "
    # We loop through the skills to format them nicely
    for i in range(len(state["skills"])):
        if i == len(state["skills"])-1:
            state["result"] = state["result"] + f"and {state["skills"][i]}."
        else:
            state["result"] = state["result"] + f"{state["skills"][i]}, "
    return state

### Step 3: The Assembly Line
We chain them together: First -> Second -> Third.

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("first", firstNode)
graph.add_node("second", secondNode)
graph.add_node("third", thirdNode)

graph.set_entry_point("first")
graph.add_edge("first", "second")
graph.add_edge("second", "third")
graph.set_finish_point("third")

app = graph.compile()

In [ ]:
from IPython.display import display, Image
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({"name": "Linda", "age": 31, "skills": ["Python", "Machine Learning", "LangGraph"]})
print(result["result"])